In [1]:
import pandas as pd
from sklearn import tree

In [12]:
events_data = pd.read_csv('https://stepik.org/media/attachments/course/4852/event_data_train.zip', compression='zip')
submission_data = pd.read_csv('https://stepik.org/media/attachments/course/4852/submissions_data_train.zip', compression='zip')
#bad data
#events_data = pd.read_csv('events_data_test.csv')
#submission_data = pd.read_csv('submission_data_test.csv')

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [5]:
submission_data.head()

,step_id,timestamp,submission_status,user_id
0,31971,1526800961,wrong,24370
1,31971,1526800976,wrong,24370
2,31971,1526800993,wrong,24370
3,31971,1526801054,correct,24370
4,31972,1526800664,wrong,24370


In [6]:
events_data.head()

,step_id,timestamp,action,user_id
0,30456,1526893787,viewed,24417
1,30456,1526893797,viewed,24417
2,30456,1526893954,viewed,24417
3,30456,1526895780,viewed,24417
4,30456,1526893787,discovered,24417


In [7]:
# перевод timestamp to datetime
events_data['date'] = pd.to_datetime(events_data.timestamp, unit = 's')
events_data['day'] = events_data.date.dt.date

submission_data['date'] = pd.to_datetime(submission_data.timestamp, unit = 's')
submission_data['day'] = submission_data.date.dt.date

In [8]:
events_data.head()

,step_id,timestamp,action,user_id,date,day
0,30456,1526893787,viewed,24417,2018-05-21 09:09:47,2018-05-21
1,30456,1526893797,viewed,24417,2018-05-21 09:09:57,2018-05-21
2,30456,1526893954,viewed,24417,2018-05-21 09:12:34,2018-05-21
3,30456,1526895780,viewed,24417,2018-05-21 09:43:00,2018-05-21
4,30456,1526893787,discovered,24417,2018-05-21 09:09:47,2018-05-21


In [9]:
days=events_data.groupby('user_id').day.nunique()

In [10]:
days.nunique()

3

In [11]:
submission_data.head()

,step_id,timestamp,submission_status,user_id,date,day
0,31971,1526800961,wrong,24370,2018-05-20 07:22:41,2018-05-20
1,31971,1526800976,wrong,24370,2018-05-20 07:22:56,2018-05-20
2,31971,1526800993,wrong,24370,2018-05-20 07:23:13,2018-05-20
3,31971,1526801054,correct,24370,2018-05-20 07:24:14,2018-05-20
4,31972,1526800664,wrong,24370,2018-05-20 07:17:44,2018-05-20


In [ ]:
# последний заход в онлайн пользователя
users_data = events_data.groupby('user_id', as_index = False)\
                    .agg({'timestamp' : 'max'})\
                    .rename({'timestamp' : 'last_timestamp'},axis =1)

In [ ]:
users_data.head()

In [ ]:
now=users_data.last_timestamp.max()

In [ ]:
drop_out_threasold = 60*60*24*30

In [ ]:
users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threasold

In [ ]:
users_data.head()

In [ ]:
# таблица из количества верныъ\неверных ответов
users_scores = submission_data.pivot_table(index = 'user_id', \
                                          columns = 'submission_status', \
                                          values = 'step_id', \
                                           aggfunc = 'count',\
                                          fill_value = 0)\
                                          .reset_index()

In [ ]:
users_scores.head(10)

In [ ]:
# объединение таблиц без потери
users_data = users_data.merge(users_scores , on = 'user_id', how = 'outer')

In [ ]:
users_data = users_data.fillna(0)

In [ ]:
users_data.head()

In [ ]:
# узнаем сколько попыток было на каждый action
users_invent_data = events_data.pivot_table(index='user_id',\
                                           columns = 'action', \
                                           values = 'step_id',\
                                           aggfunc = 'count',\
                                           fill_value = 0).reset_index()

In [ ]:
users_invent_data.head()

In [ ]:
users_data = users_data.merge(users_invent_data, on = 'user_id' , how = 'outer')

In [ ]:
# количетсво уникальных дней пользователя
users_days = events_data.groupby('user_id').day.nunique()

In [ ]:
users_days.head()

In [ ]:
users_days.to_frame().reset_index()

In [ ]:
users_days.nunique()

In [ ]:
users_data = users_data.merge(users_days, on = 'user_id', how = 'outer')

In [ ]:
users_data.head()

In [ ]:
users_data['passed_course'] = users_data.passed > 170

In [ ]:
users_data.head()

In [ ]:
# предсказания дропнется user или нет в течении 3дней

In [ ]:
users_data.day.nunique()

In [ ]:
users_data[users_data.passed_course].day.hist()

In [ ]:
users_data[users_data.passed_course].day.median()

In [ ]:
user_min_time =events_data.groupby('user_id', as_index = False) \
.agg({'timestamp' : 'min'})\
.rename({'timestamp' : 'min_timestamp'},axis =1)

In [ ]:
user_min_time.head()

In [ ]:
users_data = users_data.merge(user_min_time,on = 'user_id', how = 'outer')

In [ ]:
users_data.head()

In [ ]:
event_data_train = pd.DataFrame()

In [ ]:
# for user_id in users_data.user_id:
#     min_user_time = users_data[users_data.user_id == user_id].min_timestamp.item()
#     time_threshold = min_user_time + 3*24*60*60
    
#     user_events_data = events_data[ (events_data.user_id == user_id) & (events_data.timestamp < time_threshold) ]
    
#     event_data_train = event_data_train.append(user_events_data)

In [ ]:
# event_data_train = 


# for user_id in users_data.user_id:
#     min_user_time = 